In [1]:
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch import optim as optim

In [2]:
transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
train_dataset=torchvision.datasets.ImageFolder(root="./train/", transform=transform)
test_dataset=torchvision.datasets.ImageFolder(root="./test/", transform=transform)

In [4]:
train_dataloader=torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader=torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [5]:
image, label=train_dataset[0]

In [6]:
image.size()

torch.Size([3, 224, 224])

In [7]:
image.shape

torch.Size([3, 224, 224])

In [8]:
image, label=test_dataset[0]

In [9]:
image.size()

torch.Size([3, 224, 224])

In [10]:
image.shape

torch.Size([3, 224, 224])

In [11]:
class_names=['Church', 'Enough or Satisfied', 'Friend', 'Love', 'Me', 'Mosque', 'Seat', 'Temple', 'You']

In [12]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, 5, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5, 1)

        self.conv3 = nn.Conv2d(64, 128, 3, 1)

        self.pool3 = nn.MaxPool2d(2, 2)

        flattened_size = 128 * 25 * 25  # = 80,000

        self.fc1 = nn.Linear(flattened_size, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 9)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [13]:
net=NeuralNet()
loss_function=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [14]:
for epoch in range(20):
    print(f"Training epoch {epoch} ....")

    running_loss=0.0

    for i, data in enumerate(train_dataloader):
        inputs, labels=data
        optimizer.zero_grad()

        outputs=net(inputs)
        loss=loss_function(outputs, labels)
        
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()

    print(f"loss {running_loss / len(train_dataloader):4f}")

Training epoch 0 ....
loss 2.197755
Training epoch 1 ....
loss 2.195818
Training epoch 2 ....
loss 2.192902
Training epoch 3 ....
loss 2.187945
Training epoch 4 ....
loss 2.173939
Training epoch 5 ....
loss 2.137274
Training epoch 6 ....
loss 2.076386
Training epoch 7 ....
loss 1.977762
Training epoch 8 ....
loss 1.875980
Training epoch 9 ....
loss 1.804643
Training epoch 10 ....
loss 1.723851
Training epoch 11 ....
loss 1.646193
Training epoch 12 ....
loss 1.559615
Training epoch 13 ....
loss 1.456020
Training epoch 14 ....
loss 1.354236
Training epoch 15 ....
loss 1.206631
Training epoch 16 ....
loss 1.033800
Training epoch 17 ....
loss 0.838562
Training epoch 18 ....
loss 0.618988
Training epoch 19 ....
loss 0.415275


In [15]:
torch.save(net.state_dict(), 'model.pth')

In [16]:
net.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [17]:
correct=0
total=0

net.eval()

with torch.no_grad():
    for data in test_dataloader:
        images, labels=data
        outputs=net(images)
        _, predicted=torch.max(outputs, 1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()

accuracy=100*correct/total

print(f"Accuracy {accuracy}%")

Accuracy 40.48%


In [18]:
from PIL import Image

In [19]:
def load_image(image_path):
    image=Image.open(image_path)
    image=transform(image)
    image=image.unsqueeze(0)
    return image
image_paths=['C:\\Users\\kanyu\\projects\\signlanguage\\sol\\ImageID_0SFM7Q90.jpg']
images=[load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output=net(image)
        _, predicted=torch.max(output, 1)
        print(f"Prediction: {class_names[predicted.item()]}")


Prediction: You
